In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

from pathlib import Path
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

import spikesorters as ss
from sorter import SpikeSession
from spikeextractors import MEArecRecordingExtractor, MEArecSortingExtractor
from utils import prepare_fp_dataset

matplotlib.use('qt4Agg')

/home/mclancy/.local/share/pyenv/versions/3.8.3/lib/python3.8/importlib/__init__.py:127: MatplotlibDeprecationWarning: 
The matplotlib.backends.backend_qt4agg backend was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
tmp_path = Path('/home/mclancy/SpikeConfidence/.tmp/')
os.environ["TMP"] = str(tmp_path)
os.environ["TMPDIR"] = str(tmp_path)
os.environ["TEMPDIR"] = str(tmp_path)
os.environ["ML_TEMPORARY_DIRECTORY"] = str(tmp_path)
tmp_path.mkdir(exist_ok=True, parents=True)

base_dir = Path('/home/mclancy/SpikeConfidence/')
analyses_dir = base_dir / 'analyses'
spike_sorter_dir = base_dir / 'spikesorters'

cache_path = Path('/data/.cache')

# kilosort2_5
ss.Kilosort2_5Sorter.set_kilosort2_5_path(spike_sorter_dir / 'Kilosort')

ss.IronClustSorter.set_ironclust_path(spike_sorter_dir / 'ironclust')

dataset_path = Path("/home/mclancy/SpikeConfidence/analyses/recordings_50cells_SqMEA_2020/recordings_50cells_SqMEA-10-15_600.0_10.0uV_21-01-2020_18-12.h5")

Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/Kilosort
Setting IRONCLUST_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/ironclust


In [3]:
# Spyking circus causes d_prime to break
sorter_names = ['mountainsort4', 'herdingspikes', 'spykingcircus']#'kilosort2_5',['ironclust','tridesclous',

metric_names = np.array(["num_spikes", "firing_rate", "presence_ratio",
                "isi_violation", "amplitude_cutoff", "snr",
                "max_drift", "cumulative_drift", "silhouette_score",
                "isolation_distance", "l_ratio",
                "nn_hit_rate", "nn_miss_rate","d_prime"])

In [4]:
# Preparing dataset
recording = MEArecRecordingExtractor(dataset_path)
gt_sorting = MEArecSortingExtractor(dataset_path)

session = SpikeSession(recording, gt_sorting, cache_path=cache_path/dataset_path.stem)
X, y = prepare_fp_dataset(session, sorter_names=['herdingspikes'], metric_names=['l_ratio'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

#Standard scalar shouldn't affect Logistic regression, but it does in this case (for mountainsort4)
model = make_pipeline(StandardScaler(), LogisticRegression())
scaler = StandardScaler().fit(X_train)

X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

# False positive classification via logistic regression
model.fit(X_train_transformed, y_train)
model_accuracy = model.score(X_test_transformed, y_test)
print(f"Full metrics model accuracy is {model_accuracy}")

Full metrics model accuracy is 0.8571428571428571
